In [ ]:
import pandas as pd
from IPython.display import display, Markdown
import matplotlib.pyplot as plt

In [ ]:
caminho_arquivo = "../../dados/dados_siape_consolidado/receita_federal/cadastro_siape_receita_federal_mes_ano-2021-2023.csv"

df_rfb = pd.read_csv(caminho_arquivo)

In [ ]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)

In [ ]:
df_rfb['ano_mes'] = df_rfb['ano'].astype(str) + '-' + df_rfb['mes'].astype(str).str.zfill(2)

In [ ]:
df_rfb["REGIME_JURIDICO"].value_counts()

In [ ]:
df_rfb[(df_rfb["REGIME_JURIDICO"] == "REGIME JURIDICO UNICO") & (df_rfb["DESCRICAO_CARGO"] == "Inválido")]

In [ ]:
df_rfb[df_rfb["DESCRICAO_CARGO"] == "Inválido"].value_counts("REGIME_JURIDICO")

In [ ]:
def dicionario_servidores_e_estagio_uorg_lotacao_ano_mes (df_rfb):

    dicionario_rfb = {}

    df_uorgs_com_estagio = df_rfb[df_rfb["REGIME_JURIDICO"].str.contains("estag", na=False, case=False)]

    uorgs_lotacao = df_uorgs_com_estagio["UORG_LOTACAO"].unique()

    contador = 0

    quantidade_uorg_lotacao = len(uorgs_lotacao)
    
    for uorg_lotacao in uorgs_lotacao:

        contador += 1

        contador_mes = 0

        df_ano_mes_com_estagio = df_rfb[
                                        (df_rfb["REGIME_JURIDICO"].str.contains("estag", na=False, case=False)) &
                                        (df_rfb["UORG_LOTACAO"] == uorg_lotacao)]
        anos_meses = df_ano_mes_com_estagio["ano_mes"].unique()
        
        for ano_mes in anos_meses: 

            contador_mes += 1

            quantidade_ano_mes = len(anos_meses)

            display(f"Processando o uorg_lotacao: {uorg_lotacao} ({contador} de {quantidade_uorg_lotacao}) e mês: {ano_mes} ({contador_mes} de {quantidade_ano_mes}).")
            
            qtd_regime_unico = df_rfb[
                        df_rfb["REGIME_JURIDICO"].str.contains("regime juridico unico", na=False, case=False) &
                        (df_rfb["UORG_LOTACAO"] == uorg_lotacao) &
                        (df_rfb['ano_mes'] == ano_mes)]["Id_SERVIDOR_PORTAL"].count()
            
            qtd_estagiario =  df_rfb[
                        df_rfb["REGIME_JURIDICO"].str.contains("estag", na=False, case=False) &
                        (df_rfb["UORG_LOTACAO"] == uorg_lotacao) &
                        (df_rfb['ano_mes'] == ano_mes)]["Id_SERVIDOR_PORTAL"].count()
        
            if not uorg_lotacao in dicionario_rfb:
                dicionario_rfb[uorg_lotacao] = {}

            dicionario_rfb[uorg_lotacao][ano_mes] = {'regime_unico': qtd_regime_unico,'estagiarios': qtd_estagiario,'indice_estagio': qtd_estagiario/qtd_regime_unico}
    
    # Convertendo o dicionário em DataFrame
    df_resultado = pd.DataFrame.from_dict({(i, j): dicionario_rfb[i][j] 
                                           for i in dicionario_rfb.keys() 
                                           for j in dicionario_rfb[i].keys()},
                                          orient='index')

    df_resultado = df_resultado.reset_index().rename(columns={'level_0': 'uorg_lotacao', 'level_1': 'ano_mes'})
    
    return df_resultado

In [ ]:
df_totais = dicionario_servidores_e_estagio_uorg_lotacao_ano_mes(df_rfb)

In [ ]:
def gerar_grafico_indice_estagio (df, titulo):

    display (Markdown(f'# {titulo}'))
    
    # Criar gráfico de linha
    plt.figure(figsize=(10, 6))
    
    for org in df['uorg_lotacao'].unique():
        df_org = df[df['uorg_lotacao'] == org]
        x = df_org['ano_mes'].to_numpy()
        y = df_org['indice_estagio'].to_numpy()
        plt.plot(x, y, label=org)
    
    plt.title('Índice de Estágio ao longo do Tempo')
    plt.xlabel('Ano Mês')
    plt.ylabel('Índice de Estágio')
    plt.legend()
    plt.show()

In [ ]:
df_totais.to_csv('../../dados/dados_siape_consolidado/receita_federal/receita_federal_uorg_lotacao_com_estagio_2021_2023.csv', index=False)

In [58]:
df_totais.sort_values("estagiarios", ascending=False)[0:25]

,uorg_lotacao,ano_mes,regime_unico,estagiarios,indice_estagio
135,ALF/FOZ/ALF RFB EM FOZ DO IGUACU,2023-09,259,7,0.027027
134,ALF/FOZ/ALF RFB EM FOZ DO IGUACU,2023-08,232,7,0.030172
133,ALF/FOZ/ALF RFB EM FOZ DO IGUACU,2023-07,233,7,0.030043
143,ALF/FOZ/ALF RFB EM FOZ DO IGUACU,2023-06,236,7,0.029661
136,ALF/FOZ/ALF RFB EM FOZ DO IGUACU,2023-10,227,6,0.026432
142,ALF/FOZ/ALF RFB EM FOZ DO IGUACU,2023-05,240,6,0.025000
137,ALF/FOZ/ALF RFB EM FOZ DO IGUACU,2023-11,221,6,0.027149
836,DRF/MBA/DELEGACIA RFB EM MARABA,2021-08,35,6,0.171429
837,DRF/MBA/DELEGACIA RFB EM MARABA,2021-09,36,6,0.166667
154,ALF/FOZ/ALF RFB EM FOZ DO IGUACU,2021-05,247,5,0.020243


In [75]:
df_contagem = df_totais.groupby("uorg_lotacao").mean("estagiario")

In [80]:
df_totais.groupby("uorg_lotacao").size().sort_values(ascending=False)

uorg_lotacao
Sem informação                              35
ALF/FOZ/ALF RFB EM FOZ DO IGUACU            35
DRF/CRU/DELEGACIA RFB EM CARUARU            35
DRF/PEL/DELEGACIA RFB EM PELOTAS            35
DRF/LIM/DELEGACIA RFB EM LIMEIRA            35
DRF/JOA/DELEGACIA RFB EM JOACABA            35
SRRF/3RF/SUPERINTENDENCIA REG RFB 3A RF     34
ALF/URA/ALF RFB EM URUGUAIANA               33
DRF/OSA/DELEGACIA RFB EM OSASCO             32
ALF/COR/ALF RFB EM CORUMBA                  32
DRF/TSA/DELEGACIA RFB EM TERESINA           32
DERAT/SP/DELEGACIA ESP RFB ADM TRIBUT SP    31
ALF/VCP/ALF RFB NO AER INT VIRACOPOS        31
SRRF/8RF/SUPERINTENDENCIA REG RFB 8A RF     29
DRF/CXL/DELEGACIA RFB EM CAXIAS DO SUL      29
ALF/SPO/ALF RFB EM SAO PAULO                27
DRF/PPE/DELEGACIA RFB EM PRESIDENTE PRUD    26
DRF/UBL/DELEGACIA RFB EM UBERLANDIA         26
DRF/NAT/DELEGACIA RFB EM NATAL              26
SRRF/2RF/SUPERINTENDENCIA REG RFB 2A RF     25
DRF/NHO/DELEGACIA RFB NOVO HAMBURGO         25


In [84]:
df_sem_informacao = df_rfb[df_rfb['UORG_LOTACAO'].str.contains('Sem informação', case=False, na=False)]

In [88]:
display(Markdown('# Todas Uorgs rfb'))
display(df_rfb.value_counts('SITUACAO_VINCULO'))
display(Markdown('# Uorgs sem informação'))
display(df_sem_informacao.value_counts('SITUACAO_VINCULO'))

# Todas Uorgs rfb

SITUACAO_VINCULO
ATIVO PERMANENTE        704174
ATIVO EM OUTRO ORGAO     23275
CEDIDO/REQUISITADO        8066
EMPREGADO SERPRO          2970
CLT ANS -DEC 6657/08      1826
ESTAGIARIO SIGEPE         1641
CELETISTA DEC.JUDIC.      1024
NOMEADO CARGO COMIS.       551
EXERC. 7  ART93 8112       410
ATIVO - DEC. JUDIC         344
REQ.DE OUTROS ORGAOS       302
CONSELHEIRO DO CARF        241
EXCEDENTE A LOTACAO        194
EXERC DESCENT CARREI        38
CELETISTA                   15
SEM VINCULO                  3
EXERC. DESCENT. PGFN         2
EXERC. LEI 13681/18          1
Name: count, dtype: int64

# Uorgs sem informação

SITUACAO_VINCULO
CEDIDO/REQUISITADO      1386
ATIVO EM OUTRO ORGAO    1050
NOMEADO CARGO COMIS.     514
EXERC. 7  ART93 8112     314
REQ.DE OUTROS ORGAOS     302
ESTAGIARIO SIGEPE         35
ATIVO PERMANENTE           4
Name: count, dtype: int64

In [76]:
display(df_contagem)

,regime_unico,estagiarios,indice_estagio
uorg_lotacao,,,
ALF/BHE/ALF RFB EM BELO HORIZONTE,120.125000,1.000000,0.008345
ALF/BSB/ALF RFB AER INT BRASILIA,84.680000,1.000000,0.011826
ALF/COR/ALF RFB EM CORUMBA,41.250000,1.406250,0.033849
ALF/DCA/ALF RFB EM DIONISIO CERQUEIRA,30.142857,2.333333,0.077509
ALF/FNS/ALF RFB EM FLORIANOPOLIS,76.500000,1.000000,0.013072
ALF/FOZ/ALF RFB EM FOZ DO IGUACU,236.542857,4.457143,0.018841
ALF/GRU/ALF RFB AER INT GUARULHOS,217.200000,1.000000,0.004606
ALF/PPA/ALF RFB EM PONTA PORA,54.818182,1.454545,0.026389
ALF/SDR/ALF RFB EM SALVADOR,92.550000,2.400000,0.026015


In [ ]:
gerar_grafico_indice_estagio (df_totais, "Índice estagiários/servidores nas unidades da RFB")

In [ ]:
df_estagio = df_rfb[df_rfb["REGIME_JURIDICO"].str.contains("estag", case=False, na=False)]

display(df_estagio)